In [10]:
import numpy as np
import xarray as xr
import pandas as pd
import h5py
import os,json, sys
from pprint import pprint
sys.path.append('/Users/ellen.buckley/icepyx/')
from icepyx import icesat2data as ipd

SyntaxError: invalid syntax (icesat2data.py, line 571)

### This is just a short example. Check the full subsetting notebook below for more details. 
https://github.com/icesat2py/icepyx/blob/master/doc/examples/ICESat-2_DAAC_DataAccess2_Subsetting.ipynb

### Initialize the data request
Provide the product name, bounding box (lon/lat of the lower left and upper right corner in lat/lon grid), date, and time

In [5]:
atl07 = ipd.Icesat2Data('ATL07',[-170, 70, -130, 80],['2019-02-22','2019-02-23'], \
                           start_time='00:00:00', end_time='23:59:59')

NameError: name 'ipd' is not defined

In [ ]:
atl07.earthdata_login('user name','email')

### Use the following command to show the variable dictionary. 
Set `dictview=False` to show in the group directory view. 

In [ ]:
atl07.show_custom_options(dictview=True)

### List all the available variables but not the actual paths for simplicity

In [ ]:
atl07.order_vars.avail(options=True)

### Add a default list of variables to the data request

In [ ]:
atl07.order_vars.append(defaults=True)
pprint(atl07.order_vars.wanted)

## Example: build the list provided by Ellen as the user defined list of variables

* Group: `gtx/sea_ice_segments`
 * `latitude`
 * `longitude`
 * `geoseg_beg`
 * `geoseg_end`
* Group: `gtx/sea_ice_segments/heights`
 * `height_segment_height`
 * `height_segment_confidence`
 * `height_segment_surface_error_est`
 * `height_segment_type`
 * `height_segment_ssh_flag`
 * `height_segment_w_gaussian`
* Group: `/gtx/sea_ice_segments/stats`
 * `photon_rate`
 * `cloud_flag_asr`
 * `exmax_mean_1`
 * `exmax_mean_2`
 * `height_coarse_stdev`
 * `n_photon_used`
 * `trim_height_bottom`
 * `trim_height_top`
 * `sc_orient`

### List of predefined list of default variables

In [ ]:
pprint(atl07.order_vars.wanted.keys())

### Remove the variables not in the user specified list 

In [ ]:
atl07.order_vars.remove(var_list=['height_segment_length_seg','height_segment_quality','seg_dist_x'])
pprint(atl07.order_vars.wanted)

### Add missing variables

In [ ]:
atl07.order_vars.append(var_list=[
    'geoseg_beg','geoseg_end',
    'height_segment_surface_error_est','height_segment_w_gaussian',
    'photon_rate','cloud_flag_asr','exmax_mean_1','exmax_mean_2',
    'height_coarse_stdev','n_photons_used','trim_height_bottom',
    'trim_height_top'])
pprint(atl07.order_vars.wanted)

### Refine to remove weak beams

In [ ]:
atl07.order_vars.remove(beam_list=['gt1r','gt2r','gt3r'])
pprint(atl07.order_vars.wanted)

#### Comments
* It is much easier than the example with ATL09, because no data with the same variable name under different beams/subdirectories in ATL07. 
* ATL10 will be very different. In v02, over 30 `latitude`. If we want to filter what we need, combination of `beam_list` and `keyword_list` is needed. Check the main subsetting notebook examples. 
* Need to run append/remove a couple of times to get what you need, but still way faster than doing this by hand. 

## Pass the variable list to order data.

In [ ]:
atl07.order_granules(Coverage=atl07.order_vars.wanted)

#### A list of `orderIDs` will be saved in `atl07.granules.orderIDs`. Save this list to restart your download if it gets interrupted. 

In [ ]:
atl07.granules.orderIDs

In [ ]:
atl07.download_granules('down')

In [ ]:
import shutil
old_dir = 'down'
new_dir = 'ATL07/example/'
if not os.path.exists(new_dir): os.makedirs(new_dir)
for root, dirs, files in os.walk(old_dir, topdown=False):
    for file in files:
        try:
            shutil.move(os.path.join(root, file), os.path.join(new_dir,file))
        except OSError:
            pass
        
for root, dirs, files in os.walk(old_dir):
    for name in dirs:
        os.rmdir(os.path.join(root, name))

In [ ]:
varname = 'latitude'
#varname = 'sc_orient'

varlist = []
def IS2h5walk(vname, h5node):
    if isinstance(h5node, h5py.Dataset):
        varlist.append(vname)
    return 

with h5py.File(fn,'r') as h5pt:
    h5pt.visititems(IS2h5walk)